In [ ]:
!pip install rasterio
!pip install tensorflow

In [ ]:
# Cell 1: Mount Drive & Install Libraries
from google.colab import drive
drive.mount('/content/drive')

import rasterio
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Cell 2: Load Model and Latest Satellite Image
# Define the path to your new 3-band model
model_path = '/content/drive/MyDrive/deforestation_3band_model.h5'

# Define the path to the latest image you exported from GEE
latest_image_path = '/content/drive/MyDrive/deforestation_alerts/latest_satellite_image.tif'

try:
    # Load the trained 3-band model
    model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully.")

    # Load the latest satellite image
    with rasterio.open(latest_image_path) as src:
        new_image = src.read()
    print("Latest image loaded successfully.")

except FileNotFoundError:
    print("Error: Could not find the model or image file. Please check the paths in your Google Drive.")
    exit()

# Cell 3: Pre-process and Make a Prediction
def normalize_and_pad(image, tile_size=256):
    # Normalize the image data
    image = image.astype('float32') / 3000.0

    # Pad the image to be a multiple of the tile size
    padded_height = (image.shape[1] // tile_size + 1) * tile_size
    padded_width = (image.shape[2] // tile_size + 1) * tile_size

    padded_image = np.pad(image, ((0, 0), (0, padded_height - image.shape[1]), (0, padded_width - image.shape[2])), mode='constant')

    # Transpose to H, W, C for the model
    padded_image = np.transpose(padded_image, (1, 2, 0))
    return padded_image, padded_height, padded_width

# Process the new image
tile_size = 256
processed_image, padded_height, padded_width = normalize_and_pad(new_image, tile_size)

# Create tiles for prediction
input_tiles = []
for y in range(0, padded_height, tile_size):
    for x in range(0, padded_width, tile_size):
        input_tile = processed_image[y:y + tile_size, x:x + tile_size, :]
        input_tiles.append(input_tile)

input_for_prediction = np.array(input_tiles)

# Make the prediction
predictions = model.predict(input_for_prediction)
print("\nPrediction complete.")

# Reconstruct the full prediction mask
full_prediction_mask = np.zeros((padded_height, padded_width), dtype=np.uint8)
tile_index = 0
for y in range(0, padded_height, tile_size):
    for x in range(0, padded_width, tile_size):
        prediction_tile = (predictions[tile_index, :, :, 0] > 0.5).astype(np.uint8)
        full_prediction_mask[y:y + tile_size, x:x + tile_size] = prediction_tile
        tile_index += 1

# Crop the mask back to the original image size
original_height = new_image.shape[1]
original_width = new_image.shape[2]
final_mask = full_prediction_mask[:original_height, :original_width]

# Cell 4: Check for Deforestation and Provide an Alert
# Calculate the percentage of deforested pixels in the new mask
deforestation_count = np.sum(final_mask)
total_pixels = final_mask.size
deforestation_percentage = (deforestation_count / total_pixels) * 100

# Set a threshold for alerting
threshold_percentage = 0.1 # You can adjust this threshold
print(f"Deforestation detected: {deforestation_percentage:.2f}%")

if deforestation_percentage > threshold_percentage:
    print("\n🚨🚨🚨 ALERT! Deforestation Detected! 🚨🚨🚨")
else:
    print("\n✅ No significant deforestation detected.")

# Cell 5: Visualize the Results
# Visualize the latest image and the detected deforestation
def visualize_image(image):
    # Select bands for false-color composite
    rgb_image = np.transpose(image[:3, :, :], (1, 2, 0))
    # Clip and normalize for visualization
    vis_min = 100
    vis_max = 3000
    rgb_image_clipped = np.clip(rgb_image, vis_min, vis_max)
    normalized_image = (rgb_image_clipped - vis_min) / (vis_max - vis_min)
    return normalized_image

# Plot the latest image and the deforestation mask
fig, ax = plt.subplots(1, 2, figsize=(15, 7))

# Plot latest satellite image
ax[0].imshow(visualize_image(new_image))
ax[0].set_title('Latest Satellite Image', fontsize=14)
ax[0].axis('off')

# Plot the deforestation mask overlaid on the image
ax[1].imshow(visualize_image(new_image))
ax[1].imshow(final_mask, cmap='Reds', alpha=0.5) # Overlay the red mask
ax[1].set_title('Detected Deforestation', fontsize=14)
ax[1].axis('off')

plt.tight_layout()
plt.show()